In [10]:
from pantograph import Server
from lean_interact import *  # noqa: F403
from lean_interact.interface import LeanError, CommandResponse, Message
import litellm
import re
from dotenv import load_dotenv
import os
from collections import defaultdict
import asyncio

# Load .env file - call this FIRST, before accessing env vars
load_dotenv()
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key is None:
    raise Exception("Failed to load env!")

In [11]:
#how to create pantograph server
#server = await Server.create(imports=['Init', 'Mathlib'], project_path=".")

In [12]:
project = LocalProject(directory=".")
config = LeanREPLConfig(project=project, memory_hard_limit_mb = 8192)
server = LeanServer(config)
imports = ["Mathlib", "LeanSpecproof.Verification"]
def to_import_string(imports: list[str]):
    return "\n".join("import " + i for i in imports)
import_string = to_import_string(imports)

# this takes 1 to 2 minutes
pmath = server.run(Command(cmd=import_string))
server.run(PickleEnvironment(env=pmath.env, pickle_to="clean.olean"))

Lake version 5.0.0-src+2fcce72 (Lean version 4.27.0-rc1)
Current branch: HEAD
Using cache (Azure) from origin: leanprover-community/mathlib4
No files to download
Decompressing 7728 file(s)
Unpacked in 18371 ms
Completed successfully!
✔ [7744/7746] Built LeanSpecproof.Verification (32s)
⚠ [7745/7746] Built LeanSpecproof (11s)
info: LeanSpecproof.lean:6:0: Loogle Search Results
  [apply] #check add_comm --  {G : Type u_1} [AddCommMagma G] (a b : G) : a + b = b + a
  
  [apply] #check Sym.append_comm --  {α : Type u_1} {n' : ℕ} (s s' : Sym α n') : s.append s' = (Sym.cast ⋯) (s'.append s)
  
  [apply] #check Sym2.add.eq_1 --  {M : Type u_4} [AddCommMagma M] : Sym2.add = Sym2.lift ⟨fun x1 x2 => x1 + x2, ⋯⟩
  
  [apply] #check Composition.reverse_append --  {n m : ℕ} (c₁ : Composition m) (c₂ : Composition n) : (c₁.append c₂).reverse = (c₂.reverse.append c₁.reverse).cast ⋯
  
  [apply] #check Polynomial.sylvester_comm --  {R : Type u_1} [Semiring R] (f g : Polynomial R) (m n : ℕ) : f.sylveste

CommandResponse(env=0)

In [56]:
SYSTEM_MESSAGE = """You are a Lean 4 theorem proving assistant. Your goal is to prove theorems while respecting user-specified constraints.

<capabilities>
- You can write Lean 4 code with tactics and proofs
- Assume Mathlib is imported, no importing anything else
- You can use `sorry` temporarily to see proof states at that point
- You can run Lean commands like `#loogle`, `#check` to inspect definitions
- You can also run tactic searchers like `apply?`, `simp?`, and `rw?`.
- Command outputs will be truncated after commands with long outputs like `apply?`
- You can use all tactics like aesop, simp, grind, ring, etc.
- You will receive feedback after each attempt showing:
  * Compilation errors (if any)
  * Proof states at each `sorry`
  * Output from `#loogle`, `rw?`, and other commands
  * Verification results (whether constraints were violated)
</capabilities>

<constraints>
- You MUST NOT leave any `sorry` in your final proof
- Always update your proof at every stage (even if it fails to compile, you will be given info)
- You MUST follow the forbidden pattern constraint provided by the user
- You have {max_attempts} attempts to produce a valid proof
</constraints>

<output_format>
Structure your response using these XML tags:

<THOUGHTS>
Your reasoning about the problem and proof strategy.
Explain which theorems/tactics you plan to use and why they don't violate constraints.
</THOUGHTS>

<SOLUTION>
theorem challenge ... := by
  -- your proof here
#check useful_lemma  -- optional: check available theorems
</SOLUTION>

If you encounter an issue you cannot resolve (e.g., malformed input, impossible constraint):
<ERROR>
Brief explanation of the issue
</ERROR>
</output_format>

<example>
User gives: "Prove ∀ n : Nat, n + 0 = n without using theorems matching 'add_zero'"

Good response:
<THOUGHTS>
I need to prove n + 0 = n. The forbidden pattern is 'add_zero', so I can't use Nat.add_zero or similar.
I'll use induction on n. Base case: 0 + 0 = 0 follows from definition.
Inductive case: (n+1) + 0 = (n + 0) + 1 by definition, then use IH.
</THOUGHTS>

<SOLUTION>
theorem challenge (n : Nat) : n + 0 = n := by
  induction n with
  | zero => rfl
  | succ n ih => 
    rw [Nat.add_succ]
#check Nat.add_succ
</SOLUTION>
</example>

<advice>
SEARCH STRATEGY:
- Use #loogle "pattern" to find theorems (NOT #check)
- If #check fails on a name from #loogle results, try: #check @TheoremName or open the namespace
- Stop searching after 3 failed attempts - try a different proof approach instead
- Batch searches with proof attempts: search AND try tactics simultaneously
- You can chain 20+ commands in a single query: leverage this and batch all loogle searches together
- Try to finish in as few attempts as possible

<example>
theorem challenge (m n : ℕ) : (m<...=n).toList = if m + 1 ≤ n then (m + 1) :: ((m + 1)<...=n).toList else []
Attempt 1:
<THOUGHTS>
I need to prove that `(m<...=n).toList = if m + 1 ≤ n then (m + 1) :: ((m + 1)<...=n).toList else []` without using any theorem containing "toList_roc_eq_if".

First, let me understand what `<...=` means - this looks like the "range of consecutive" notation, likely `Ico` (interval closed-open) or similar.

I am working with natural numbers, let me search for relevant theorems.
While searching, since there is an if case, let me split the goal based on if m + 1 ≤ n and try to close via simp_all.
</THOUGHTS>
<SOLUTION>
#loogle Nat, "toList_roc_eq"
#loogle Nat, "toList_roc", "cons"
#loogle Nat, "roc", _ = _ :: _

#loogle Std.Roc, "toList", "eq"
#loogle Std.Roc, "toList", "roc"
#loogle Std.Roc, "toList"

#loogle Std.Roc, ⊢ _.toList = _
#loogle (?m<...=?n).toList = _
#loogle Nat, "toList_roc", "iff"
#loogle Nat, Std.Roc, "eq"
#loogle Nat, "roc", "nil"
#loogle Nat, Std.Roc, "toList"

#loogle "roc"
#loogle Nat, "roc"
#loogle Nat, (_<...=_).toList = _
theorem challenge (m n : ℕ) : (m<...=n).toList = if m + 1 ≤ n then (m + 1) :: ((m + 1)<...=n).toList else [] := by
  by_cases h : m + 1 ≤ n
  · simp_all
  · simp_all
</SOLUTION>
</example>
</advice>
<loogle>
Loogle Usage

Loogle finds definitions and lemmas in various ways:

By constant:
🔍 Real.sin
finds all lemmas whose statement somehow mentions the sine function.

By lemma name substring:
🔍 "differ"
finds all lemmas that have "differ" somewhere in their lemma name.

By subexpression:
🔍 _ * (_ ^ _)
finds all lemmas whose statements somewhere include a product where the second argument is raised to some power.

The pattern can also be non-linear, as in
🔍 Real.sqrt ?a * Real.sqrt ?a

If the pattern has parameters, they are matched in any order. Both of these will find List.map:
🔍 (?a -> ?b) -> List ?a -> List ?b
🔍 List ?a -> (?a -> ?b) -> List ?b

By main conclusion:
🔍 ⊢ tsum _ = _ * tsum _
finds all lemmas where the conclusion (the subexpression to the right of all → and ∀) has the given shape.

As before, if the pattern has parameters, they are matched against the hypotheses of the lemma in any order; for example,
🔍 ⊢ _ < _ → tsum _ < tsum _
will find tsum_lt_tsum even though the hypothesis f i < g i is not the last.

If you pass more than one such search filter, separated by commas Loogle will return lemmas which match all of them. The search
🔍 Real.sin, "two", tsum, _ * _, _ ^ _, ⊢ _ < _ → _
woould find all lemmas which mention the constants Real.sin and tsum, have "two" as a substring of the lemma name, include a product and a power somewhere in the type, and have a hypothesis of the form _ < _ (if there were any such lemmas). Metavariables (?a) are assigned independently in each filter.

Loogle results will be truncated, so search specifically. 

<loogle_specificity_rules>
**ALWAYS BE AS SPECIFIC AS POSSIBLE WITH LOOGLE SEARCHES**
Rule 1: ALWAYS include the type if you know it
- BAD:  #loogle "add_comm"
- GOOD: #loogle Nat, "add_comm"
- BEST: #loogle Nat, "add_comm", "succ"

Rule 2: Include BOTH type AND structure
- BAD:  #loogle "toList"
- GOOD: #loogle Std.Roc, "toList"  

Rule 3: Prefer multiple specific searches over one generic search
- Instead of: #loogle "toList"
- Do this:    #loogle Nat, "toList"
              #loogle List, "toList"
              #loogle Array, "toList"
Rule 4: Never reference variables in the theorem, use generic patterns
- BAD: #loogle Nat, (m <...= n).toList = _
- GOOD: #loogle Nat, (?m <...= ?n).toList = _
- GOOD: #loogle Nat, (_ <...= _).toList = _
</loogle_specificity_rules>
<loogle_search_rules>
**ALWAYS HAVE MULTIPLE SEARCHES, SOME BROAD, MOST SPECIFIC**
**ALWAYS HAVE MULTIPLE TYPES OF SEARCHES, INCLUDING PATTERNS AND DIFFERENT THEOREM NAMES**
</loogle_search_rules>
</loogle>
"""
def create_user_message(theorem_code: str, forbidden_pattern: str, attempt: int, max_attempts: int) -> str:
    return f"""<task>
Prove the following theorem WITHOUT using any theorem whose name contains "{forbidden_pattern}".

This is attempt {attempt}/{max_attempts}.

{theorem_code}
</task>"""


In [57]:
SPAM_MARKERS = ["Loogle Usage"]
def is_spam(msg_text: str) -> bool:
   return any(spam in msg_text for spam in SPAM_MARKERS)

In [58]:
def format_lean_output(response: CommandResponse | LeanError, base_code: str, header_lines: int,
                       max_line: int = 5, max_size: int = 250) -> str:
    if isinstance(response, LeanError):
        return f"<fatal>\n Fatal Lean Error: {response.message}\n</fatal>"
    parts = []
    errors = defaultdict(lambda:[])
    warnings = defaultdict(lambda:[])
    infos = defaultdict(lambda:[])
    lines = base_code.split("\n")
    for msg in response.messages:
        if is_spam(msg.data):
            continue
        if msg.severity == 'error':
            errors[msg.start_pos.line].append(msg)
        if msg.severity == "info":
            infos[msg.start_pos.line].append(msg)
        if msg.severity == "warning":
            warnings[msg.start_pos.line].append(msg)
    def format_sl(msg: Message):
        ln = msg.start_pos.line - header_lines
        if ln >= len(lines):
            return [msg.data]
        else:
            return [f"Line {ln}: " +
                    f"{lines[ln - 1].lstrip()} →",
                    msg.data[:max_size]]
    # Extract errors
    if errors:
        parts.append("<errors>")
        for err_line in errors:
            for err in errors[err_line][:max_line]:
                parts += format_sl(err)
        parts.append("</errors>")
    if warnings:
        parts.append("<warnings>")
        for warn_line in warnings:
            for warn in warnings[warn_line][:max_line]:
                parts += format_sl(warn)
        parts.append("</warnings>")
    if infos:
        parts.append("<info>")
        for info_line in infos:
            for info in infos[info_line][:max_line]:
                parts += format_sl(info)
        parts.append("</info>")
        
    
    # Extract sorry states
    if response.sorries:
        parts.append("<proof_states>")
        for i, sorry in enumerate(response.sorries, 1):
            parts.append(f"Sorry #{i} at line {sorry.start_pos.line - header_lines}:")
            parts.append(sorry.goal)
        parts.append("</proof_states>")
    
    return "\n".join(parts)

In [59]:
async def prove_theorem_loop(
    theorem_code: str,
    forbidden_pattern: str,
    max_attempts: int = 5,
    model: str = "anthropic/claude-sonnet-4-20250514",
    imports: list[str] = imports,
    verbose: bool = False,
    hint: str = None,
    sleep: float = None
) -> dict:
    """
    Main loop for theorem proving with Claude
    
    Returns dict with:
        - success: bool
        - final_code: str (if success)
        - attempt_count: int
        - messages: list (full conversation history)
    """

    # Initialize messages
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE.format(max_attempts=max_attempts)},
        {"role": "user", "content": create_user_message(theorem_code, forbidden_pattern, 1, max_attempts)}
    ]
    if hint is not None:
        messages.append({"role": "user", "content": hint})
    
    for attempt in range(1, max_attempts + 1):
        if (verbose):
            print(f"\n=== Attempt {attempt}/{max_attempts} ===")
        # Get Claude's response
        response = await litellm.acompletion(
            model=model,
            messages=messages,
            temperature=0.1
        )
        assistant_msg = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_msg})
        
        if (verbose):
            print(f"Claude response:\n{assistant_msg}\n")
        
        # Check if Claude gave up
        if "<ERROR>" in assistant_msg:
            return {
                "success": False,
                "error": "Claude reported an error",
                "attempt_count": attempt,
                "messages": messages
            }
        
        # Extract code from <SOLUTION> tags
        solution_match = re.search(r'<SOLUTION>\s*(.*?)\s*</SOLUTION>', assistant_msg, re.DOTALL)
        if not solution_match:
            feedback = "<feedback>\n<error>No <SOLUTION> tag found in response</error>\n</feedback>"
            messages.append({"role": "user", "content": feedback})
            continue
        
        lean_code = solution_match.group(1).strip()
        final_code = (to_import_string(imports) + "\n" + lean_code + "\n" +
                      f"#verify_solution challenge forbids_pattern {forbidden_pattern}")
        
        # Run through Lean
        lean_response = server.run(Command(cmd=final_code))
        
        # Format the output
        formatted_output = format_lean_output(lean_response, lean_code, len(imports))
        
        if verbose:
            print(f"Lean output:\n{formatted_output}\n")
        
        # Check for success
        valid = lean_response.lean_code_is_valid(allow_sorry=False)
        
        if valid:
            # Success!
            return {
                "success": True,
                "final_code": lean_code,
                "attempt_count": attempt,
                "messages": messages
            }
        
        # Prepare feedback for next iteration
        feedback = f"<feedback>\nAttempt {attempt}/{max_attempts} failed.\n{formatted_output}\n</feedback>"
        messages.append({"role": "user", "content": feedback})

        await asyncio.sleep(sleep)
    
    # All attempts exhausted
    return {
        "success": False,
        "error": "Maximum attempts reached",
        "final_code": lean_code,
        "attempt_count": max_attempts,
        "messages": messages
    }


In [60]:
ans2 = await prove_theorem_loop(theorem_code = "∀ {m n : ℕ}, (m<...=n).toList = if m + 1 ≤ n then (m + 1) :: ((m + 1)<...=n).toList else []",
                         forbidden_pattern = "toList_roc_eq_if",
                         max_attempts = 15,
                         sleep = 10,
                         verbose = True)


=== Attempt 1/15 ===
Claude response:
<THOUGHTS>
I need to prove that `(m<...=n).toList = if m + 1 ≤ n then (m + 1) :: ((m + 1)<...=n).toList else []` without using any theorem containing "toList_roc_eq_if".

First, let me understand what `<...=` means - this looks like the "range of consecutive" notation, likely `Ico` (interval closed-open) or similar from Std.Range or similar.

Let me search for relevant theorems about this notation and toList, while also trying a case split approach since there's an if-then-else in the goal.
</THOUGHTS>

<SOLUTION>
#loogle Std.Range, "toList"
#loogle Nat, "roc", "toList"
#loogle Std.Roc, "toList"
#loogle (_ <...= _).toList = _
#loogle Nat, (_ <...= _).toList
#loogle "roc", "toList", "cons"
#loogle "roc", "toList", "nil"
#loogle Std.Range, "toList", "cons"

theorem challenge {m n : ℕ} : (m<...=n).toList = if m + 1 ≤ n then (m + 1) :: ((m + 1)<...=n).toList else [] := by
  by_cases h : m + 1 ≤ n
  · simp [h]
    sorry
  · simp [h]
    sorry
</SOLUTIO

In [61]:
ans2

{'success': True,
 'final_code': '#check Nat.add_one_le_iff\n#check Nat.le_of_not_lt\n\ntheorem challenge {m n : ℕ} : (m<...=n).toList = if m + 1 ≤ n then (m + 1) :: ((m + 1)<...=n).toList else [] := by\n  by_cases h : m + 1 ≤ n\n  · -- Case: m + 1 ≤ n\n    simp only [h, if_true]\n    -- Need to show (m<...=n).toList = (m + 1) :: ((m + 1)<...=n).toList\n    -- This follows from Nat.toList_roc_eq_cons if we can show m < n\n    have hlt : m < n := Nat.add_one_le_iff.mp h\n    exact Nat.toList_roc_eq_cons hlt\n  · -- Case: ¬m + 1 ≤ n\n    simp only [h, if_false]\n    -- Need to show (m<...=n).toList = []\n    -- This follows from Nat.toList_roc_eq_nil if we can show n ≤ m\n    have hle : n ≤ m := by\n      rw [Nat.add_one_le_iff] at h\n      exact Nat.le_of_not_lt h\n    exact Nat.toList_roc_eq_nil hle',
 'attempt_count': 5,
 'messages': [{'role': 'system',
   'content': 'You are a Lean 4 theorem proving assistant. Your goal is to prove theorems while respecting user-specified constraints

In [22]:
lean_code = """#loogle "toList_roc_rcc"""
forbidden_pattern = "toList_roc_eq_if"
rsp = server.run(Command(cmd=((to_import_string(imports) + "\n" + lean_code + "\n" +
                      f"#loogle rio"))))
print(format_lean_output(rsp, lean_code, len(imports)))

<errors>
unterminated string literal
</errors>
<warnings>
Loogle search failed with error: unknown identifier 'rio'
</warnings>
<info>
Loogle Usage

Loogle finds definitions and lemmas in various ways:

By constant:
🔍 Real.sin
finds all lemmas whose statement somehow mentions the sine function.

By lemma name substring:
🔍 "differ"
finds all lemmas that have "differ" somewhere in their lemma name.

By subexpression:
🔍 _ * (_ ^ _)
finds all lemmas whose statements somewhere include a product where the second argument is raised to some power.

The pattern can also be non-linear, as in
🔍 Real.sqrt ?a * Real.sqrt ?a

If the pattern has parameters, they are matched in any order. Both of these will find List.map:
🔍 (?a -> ?b) -> List ?a -> List ?b
🔍 List ?a -> (?a -> ?b) -> List ?b

By main conclusion:
🔍 |- tsum _ = _ * tsum _
finds all lemmas where the conclusion (the subexpression to the right of all → and ∀) has the given shape.

As before, if the pattern has parameters, they are matched a

In [28]:
ans["messages"]

[{'role': 'system',
  'content': 'You are a Lean 4 theorem proving assistant. Your goal is to prove theorems while respecting user-specified constraints.\n\n<capabilities>\n- You can write Lean 4 code with tactics and proofs\n- Assume Mathlib is imported, no importing anything else\n- You can use `sorry` temporarily to see proof states at that point\n- You can run Lean commands like `#check`, `#loogle` to inspect definitions\n- You can also run tactic searchers like `apply?`, `simp?`, and `rw?`.\n- Command outputs will be truncated after commands like `apply?`\n- You will receive feedback after each attempt showing:\n  * Compilation errors (if any)\n  * Proof states at each `sorry`\n  * Output from `#check`, `rw?`, and other commands\n  * Verification results (whether constraints were violated)\n</capabilities>\n\n<constraints>\n- You MUST NOT leave any `sorry` in your final proof\n- You MUST follow the forbidden pattern constraint provided by the user\n- You have 15 attempts to produc